In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
from tqdm import tqdm

import plotly
# from chart_studio import plotly



In [10]:
dataframe = pd.read_csv('big_data.csv')

/var/folders/kc/jdw9593x3ksdmnhyd571zm440000gp/T/ipykernel_21422/1281625065.py:1: DtypeWarning: Columns (2,5,6,8,10,11,14,15,16,17,18,19,20,21,23,24,25,27,28,32) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv('big_data.csv')


In [11]:
#create columns that show if a person uses a product
dataframe["uses_invest"] = ~dataframe['INV_OPEN_DATE'].isna() 
dataframe["uses_money"] = ~dataframe['MONEY_ACCOUNT_OPEN_DATE'].isna()
dataframe["uses_relay"] = ~dataframe['RELAY_ORIGINAL_DATE'].isna() 
dataframe["uses_loan"] = ~dataframe['ORIGINATION_DATE'].isna()
dataframe["uses_credit"] = ~dataframe['ONBOARDING_SUCCESS_DATE'].isna()

#create column that shows how many products a person uses
dataframe["num_products"] = (dataframe[['uses_invest', 'uses_money', 'uses_relay', 'uses_loan', 'uses_credit']] == True).sum(axis=1)

# create a column that shows how long a person has used a product. If they have never used a product, it will be NaN

dataframe['date_of_creating_new_columns'] = pd.to_datetime('today').date()

dataframe['INV_OPEN_DATE'] = pd.to_datetime(dataframe['INV_OPEN_DATE']).dt.date
dataframe['days_with_invest'] = (dataframe['date_of_creating_new_columns'] - dataframe['INV_OPEN_DATE']).dt.days

dataframe['MONEY_ACCOUNT_OPEN_DATE'] = pd.to_datetime(dataframe['MONEY_ACCOUNT_OPEN_DATE']).dt.date
dataframe['days_with_money'] = (dataframe['date_of_creating_new_columns'] - dataframe['MONEY_ACCOUNT_OPEN_DATE']).dt.days

dataframe['RELAY_ORIGINAL_DATE'] = pd.to_datetime(dataframe['RELAY_ORIGINAL_DATE']).dt.date
dataframe['days_with_relay'] = (dataframe['date_of_creating_new_columns'] - dataframe['RELAY_ORIGINAL_DATE']).dt.days

dataframe['ORIGINATION_DATE'] = pd.to_datetime(dataframe['ORIGINATION_DATE']).dt.date
dataframe['days_with_loan'] = (dataframe['date_of_creating_new_columns'] - dataframe['ORIGINATION_DATE']).dt.days

dataframe['ONBOARDING_SUCCESS_DATE'] = pd.to_datetime(dataframe['ONBOARDING_SUCCESS_DATE']).dt.date
dataframe['days_with_credit'] = (dataframe['date_of_creating_new_columns'] - dataframe['ONBOARDING_SUCCESS_DATE']).dt.days



### Columns for first and second product

In [13]:
# create a column 'first_product' that shows the first product a person used, and a column 'second_product' that shows the second product a person used. If they have never used a second product, it will be NaN

dataframe['first_product'] = np.nan
dataframe['second_product'] = np.nan

first_product_list = []
second_product_list = []

for i in tqdm(range(len(dataframe))):
    #create a list of the products that the user has
    products = []
    if dataframe.loc[i, 'uses_relay'] == True:
        products.append('relay')
    if dataframe.loc[i, 'uses_money'] == True:
        products.append('money')
    if dataframe.loc[i, 'uses_invest'] == True:
        products.append('invest')
    if dataframe.loc[i, 'uses_loan'] == True:
        products.append('loan')
    if dataframe.loc[i, 'uses_credit'] == True:
        products.append('credit')

    

    #sort the list of products by the number of days the user has used the product
    products.sort(key=lambda x: dataframe.loc[i, f'days_with_{x}'], reverse=True)

    #add the first two products to the list
    if len(products) >= 1:
        first_product_list.append(products[0])
    else:
        first_product_list.append(np.nan)
    if len(products) >= 2:
        second_product_list.append(products[1])
    else:
        second_product_list.append(np.nan)
        

dataframe['first_product'] = first_product_list
dataframe['second_product'] = second_product_list




    


100%|██████████| 3216851/3216851 [04:05<00:00, 13114.35it/s]


In [16]:
#create new columns, xbuy_days, xbuy_date, first_30

# xbuy_days is the number of days between the first product a person used and the second product a person used
# xbuy_date is the date of the second product a person used

def calculate_xbuy_days(row):
    #if the person has never used a second product, return NaN
    if type(row['second_product']) == float:
        return None

    if row['first_product'] == 'relay':
        first_product_date = row['RELAY_ORIGINAL_DATE']
    elif row['first_product'] == 'money':
        first_product_date = row['MONEY_ACCOUNT_OPEN_DATE']
    elif row['first_product'] == 'invest':
        first_product_date = row['INV_OPEN_DATE']
    elif row['first_product'] == 'loan':
        first_product_date = row['ORIGINATION_DATE']
    elif row['first_product'] == 'credit':
        first_product_date = row['ONBOARDING_SUCCESS_DATE']

    if row['second_product'] == 'relay':
        second_product_date = row['RELAY_ORIGINAL_DATE']
    elif row['second_product'] == 'money':
        second_product_date = row['MONEY_ACCOUNT_OPEN_DATE']
    elif row['second_product'] == 'invest':
        second_product_date = row['INV_OPEN_DATE']
    elif row['second_product'] == 'loan':
        second_product_date = row['ORIGINATION_DATE']
    elif row['second_product'] == 'credit':
        second_product_date = row['ONBOARDING_SUCCESS_DATE']

    second_product_date = pd.to_datetime(second_product_date)
    first_product_date = pd.to_datetime(first_product_date)

    return (second_product_date - first_product_date).days


def get_xbuy_date(row):
    #gets the date the second product was adopted.
    if type(row['second_product']) == float:
        return None
    
    if row['second_product'] == 'relay':
        return row['RELAY_ORIGINAL_DATE']
    elif row['second_product'] == 'money':
        return row['MONEY_ACCOUNT_OPEN_DATE']
    elif row['second_product'] == 'invest':
        return row['INV_OPEN_DATE']
    elif row['second_product'] == 'loan':
        return row['ORIGINATION_DATE']
    elif row['second_product'] == 'credit':
        return row['ONBOARDING_SUCCESS_DATE']

    
tqdm.pandas()
dataframe['xbuy_days'] = dataframe.progress_apply(calculate_xbuy_days, axis=1)
dataframe['xbuy_date'] = dataframe.progress_apply(get_xbuy_date, axis=1)

100%|██████████| 3216851/3216851 [00:36<00:00, 88892.39it/s] 


In [14]:
#save the dataframe
dataframe.to_csv('big_data_added_columns.csv', index=False)

In [17]:
#show how many users use personal loans
dataframe['uses_loan'].value_counts()


False    2703952
True      512899
Name: uses_loan, dtype: int64